In [1]:
#To increase cell width of ipynb
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import tensorflow as tf
from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random

In [4]:
def extract_data(filepath):
    data = []
    data_labels = []
    IMG_SIZE = 100
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
    vgg = tf.contrib.keras.applications.VGG16(input_shape=IMG_SHAPE, weights="imagenet", include_top=False)
    for label in os.listdir(filepath):
        label_path = os.path.join(filepath, label)
        count = 0
        list = len(os.listdir(label_path))
        for img in os.listdir(label_path):
            image_path = os.path.join(label_path, img)
            image = cv.imread(image_path)
            re_image = cv.resize(image, (IMG_SIZE, IMG_SIZE))
            # grayImage = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
            # (thresh, BW) = cv.threshold(grayImage, 127, 255, cv.THRESH_BINARY)
            data.append(re_image)
            data_labels.append(label)

    data = np.array(data, dtype="float") / 255.0
    features = vgg.predict(data, batch_size=10)
    features_flatten = features.reshape((features.shape[0], 3 * 3 * 512))
    # data = data.reshape(data.shape[0], 50, 50, 3)
    data_labels = np.array(data_labels)
    label_names = np.unique(data_labels)
    # data_labels = tf.one_hot(indices=data_labels, depth=10)
    print(features_flatten.shape)
    return features_flatten, data_labels, label_names

In [5]:
if __name__ == "__main__":
       
    train_data_path = '../Fruit_dataset/Train'
    test_data_path = '../Fruit_dataset/Test'

    # For easier disk read operation
    np_train_path = '../Fruit_dataset/extracted_features/train.npy.gz'
    np_test_path = '../Fruit_dataset/extracted_features/test.npy.gz'
    np_train_label_path = '../Fruit_dataset/extracted_features/train_labels.npy'
    np_test_label_path = '../Fruit_dataset/extracted_features/test_labels.npy'
    np_label_name_path = '../Fruit_dataset/extracted_features/label_names.npy'

    print("\n..........loading dataset from disk..........\n")
    train_data, train_labels, label_names = extract_data(train_data_path)
    test_data, test_labels, _ = extract_data(test_data_path)

    os.makedirs(os.path.dirname(np_train_path), exist_ok=True)

    with gzip.GzipFile(np_train_path, "w") as f:
        np.save(f, train_data)
    with gzip.GzipFile(np_test_path, "w") as f:
        np.save(f, test_data)

    np.save(np_train_label_path, train_labels)
    np.save(np_test_label_path, test_labels)
    np.save(np_label_name_path, label_names)


..........loading dataset from disk..........

(6578, 4608)
